In [1]:
#Librerias:

from bs4 import BeautifulSoup
import requests
from deep_translator import GoogleTranslator
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import os

In [2]:
#Traer la página on bs4 para extraer los enlaces:

url = 'https://www.classcentral.com/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

## Se buscan los enlaces
url_todos = []
for link in soup.body.find_all("a"):
    enlace = link.get("href")
    if enlace is not None:
      url_todos.append(enlace)

In [3]:
#Se organizan los enlaces
i = 0
for enlace in url_todos:
  if enlace.startswith("/"):
    nueva_url = "https://www.classcentral.com" + enlace
    url_todos[i] = nueva_url
  i+=1

In [4]:
#Se toman los enlaces unicos
url = []
for i in url_todos:
  if i in url:
    continue
  else:
    url.append(i)

In [5]:
#Se crea diccionario para facilitar la busqueda del enlace y a qué corresponde
enlaces = {}
for enlace in url:
  valor = enlace
  dic = enlace.replace("https://www.classcentral.com/", "")
  if dic == '':
    dic = "classcentral"
    enlaces[dic] = enlace
  else:
    enlaces[dic] = enlace

In [6]:
enlaces

{'classcentral': 'https://www.classcentral.com/',
 'rankings': 'https://www.classcentral.com/rankings',
 'collections': 'https://www.classcentral.com/collections',
 'subjects': 'https://www.classcentral.com/subjects',
 'subject/cs': 'https://www.classcentral.com/subject/cs',
 'subject/health': 'https://www.classcentral.com/subject/health',
 'subject/maths': 'https://www.classcentral.com/subject/maths',
 'subject/business': 'https://www.classcentral.com/subject/business',
 'subject/humanities': 'https://www.classcentral.com/subject/humanities',
 'subject/engineering': 'https://www.classcentral.com/subject/engineering',
 'subject/science': 'https://www.classcentral.com/subject/science',
 'subject/education': 'https://www.classcentral.com/subject/education',
 'subject/social-sciences': 'https://www.classcentral.com/subject/social-sciences',
 'subject/art-and-design': 'https://www.classcentral.com/subject/art-and-design',
 'subject/data-science': 'https://www.classcentral.com/subject/data-

In [19]:
enlaces2 = {}
for enlace in url:
  valor = enlace
  valor = enlace.replace("/","_")
  dic = enlace.replace("https://www.classcentral.com/", "")
  if dic == '':
    dic = "classcentral"
    enlaces2[dic.replace("/","_")] = enlace
  else:
    enlaces2[dic.replace("/","_")] = enlace

In [20]:
enlaces2

{'classcentral': 'https://www.classcentral.com/',
 'rankings': 'https://www.classcentral.com/rankings',
 'collections': 'https://www.classcentral.com/collections',
 'subjects': 'https://www.classcentral.com/subjects',
 'subject_cs': 'https://www.classcentral.com/subject/cs',
 'subject_health': 'https://www.classcentral.com/subject/health',
 'subject_maths': 'https://www.classcentral.com/subject/maths',
 'subject_business': 'https://www.classcentral.com/subject/business',
 'subject_humanities': 'https://www.classcentral.com/subject/humanities',
 'subject_engineering': 'https://www.classcentral.com/subject/engineering',
 'subject_science': 'https://www.classcentral.com/subject/science',
 'subject_education': 'https://www.classcentral.com/subject/education',
 'subject_social-sciences': 'https://www.classcentral.com/subject/social-sciences',
 'subject_art-and-design': 'https://www.classcentral.com/subject/art-and-design',
 'subject_data-science': 'https://www.classcentral.com/subject/data-

In [21]:
#Extraer códigos fuente con selenium
driver = webdriver.Chrome()
codigo_fuente = []
for i in enlaces:
  try:
    driver.get(enlaces[i])
    time.sleep(5)

    ## Copiar el código fuente de la página
    codigo_fuente.append(driver.page_source)
  except NameError:
    print(i, NameError)
driver.close()

KeyboardInterrupt: 

In [ ]:
#Traducir y descargar todos los html:
j = 0

try:
  for codigo in codigo_fuente:
    #Transformar para capturar datos del html
    soup = BeautifulSoup(codigo, 'html.parser')

    traductor = GoogleTranslator(source='en', target='hi')
    trans = {}
    trans_dir = {}

    #Obtener texto traducido
    for child in soup.stripped_strings:
      if len(child) > 5000:
        continue
      else:
        if child == 'Copy link':
          continue
        else:
          traduccion = (traductor.translate(child))
          trans[str(child)]=str(traduccion)

    #Valores unicos para traducir      
    texto = list(soup.stripped_strings)
    texto2 = []
    for palabra in texto:
        if palabra in texto2:
            continue
        else:
            texto2.append(palabra)
    #traduccion de la pagina
    for i in range (len(texto2)):
        for texto in soup.find_all(string=True):
          if texto.parent.name != 'style' and texto.parent.name != 'script':
            if texto2[i] in trans.keys():
                nuevo_texto = texto.replace(texto2[i], trans[texto2[i]])
                texto.replace_with(nuevo_texto)

    #descargar archivo traducido:
    mensaje = str(soup)
    for name in enlaces2:
      archivo = open(name + ".html", "w", encoding="utf-8")
      archivo.write(mensaje)
      archivo.close()
      j+=1
except NameError:
    print(j, NameError)